# Working with Weak Lensing Maps

##### Spencer Everett, SLAC 2015

First import the necessary code modules.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import scipy as sp
import os,sys
import astropy.io.fits as pyfits

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import pangloss

## Convergence maps
Let's create a new `Kappamap` object `K` for a test file located in a data directory created in the local clone of the `Pangloss` repository. Note that as it is not a FITS file, the optional parameter is set to `False`.

In [4]:
K = pangloss.Kappamap(pangloss.KAPPA_FILE,FITS=False)

AttributeError: 'module' object has no attribute 'demo_data_files'

`Kappamap` is now a subclass of `WLMap` (weak-lensing map), which contains most of the methods used in the previous version of `Kappamap`. As the file is not a fits file, the following parameters are automatically set:

Field Size (self.field): 4 deg x 4 deg  

Number of Pixels (self.NX*self.NX): 4096 x 4096

Pixel-scale (self.PIXSCALE): 4 deg / 4096 = 0.000977 deg/pix

Output (self.output): 'data/GGL_los_8_0_0_N_4096_ang_4_rays_to_plane_37_f.kappa.fits'

As part of the constructor, the .kappa file is also converted and saved as a .fits file in the same directory.
Convergence values in the kappa-map are obtained using the `at()` method:

In [ ]:
print K.at(0,0), K.at(.002,-.001), K.at(np.deg2rad(1), np.deg2rad(-1))

Attempting to retrieve convergence values outside of the map always results in a return of `None` (to enable error handling).

In [ ]:
print K.at(0,np.pi)

A `Kappamap` object can also plot itself using the `plot()` method.

In [ ]:
K.plot()

The size of the plot can be changed using the first kwarg `fig_size`, where the value is in inches. The default size is 10 in.

In [ ]:
K.plot(fig_size=7)

The plot method can also be used to graph a subplot. Simply add a list parameter of the form:

    subplot = [max_RA, min_RA, min_DEC, max_DEC]

The default coordinate system is `world`, with units of degrees.

For a much more zoomed in picture:

In [ ]:
K.plot(subplot=[2.0, 1.8, -2.0, -1.8])

Alternatively, the subplot bounds can be passed with `physical` (degrees relative to the field center) or `image` coordinates (pixels) using the kwarg `coords`.

In [ ]:
K.plot(10, [-0.01, 0.01, -0.01, 0.01], coords='physical')

Note that the plot axes are always labeled using world coordinates.

Finally, while the original map itself is square (4096 x 4096 pixels) the subimage does not need to be square.

In [ ]:
K.plot(subplot=[2.0, 1.0, -2.0, -1.25])

## Shear maps
We can also create a `Shearmap` object. Like `Kappamap`, the `Shearmap` class is a subclass of `WLMap` but requires two file inputs; one file for each component of the complex shear. The files $\textit{must}$ be passed in a list (unlike `Kappamap` files, which are automatically converted) and must be in order (first the .gamma1 file, then the .gamma2 file). As before, the optional second argument is set to `FITS = False` as the file has yet to be converted to FITS format.

In [ ]:
S = pangloss.Shearmap([pangloss.GAMMA_1_FILE, pangloss.GAMMA_2_FILE], FITS=False)

A `Shearmap` object is currently two separate maps: gamma1 and gamma2. Each map attribute is now stored as a list, with the first and second entry corresponding to gamma1 and gamma2. For example, 

In [ ]:
print S.values[0] # The shearmap values for the gamma1 map

In [ ]:
S.values[1] # The shearmap values for the gamma2 map

If the files have .gamma1 and .gamma2 extensions, the attributes are set automatically in `WLMap`. A few of the important attributes set for the maps are:

Field Size (self.field): 4 deg x 4 deg  

Number of Pixels (self.NX*self.NX): 4096 x 4096

Pixel-scale (self.PIXSCALE): 4 deg / 4096 = 0.000977 deg/pix

Output (self.output): ['data/GGL_los_8_0_0_N_4096_ang_4_rays_to_plane_37_f.gamma1.fits',
'data/GGL_los_8_0_0_N_4096_ang_4_rays_to_plane_37_f.gamma1.fits']

The `at()` method works similarly for a `Shearmap`, but a third argument now specifies which map (gamma1 or gamma2) the method will extract from.

In [ ]:
print S.at(0, 0, 0) # gamma1 value at (0,0)

In [ ]:
print S.at(0, 0, 1) # gamma2 value at (0,0)

Note that if no third argument is passed, the method returns `None` (to avoid confusion).

In [ ]:
print S.at(0, 0)

The `plot()` method for a `Shearmap` is quite different. At each position on a 40x40 grid, a "shearstick" is plotted, with length proportional to the magnitude of the shear, $\vert\gamma\vert = \sqrt{\gamma_1^2 + \gamma_2^2}$, and oriented so that it lies at an angle $\phi$ counter-clockwise from the horizontal (RA) axis. (The complex shear $\gamma = \gamma_1 + i \gamma_2 = \vert\gamma\vert \left( \cos{2\phi} + i \sin{2\phi} \right)$.) 

In [ ]:
S.plot()

Resizing the image is the same as with `Kappamap`, simply pass one argument with the desired figure size in inches.

In [ ]:
S.plot(fig_size=6)

Subplots also work with `Shearmaps`. Note that the shear stick sampling for the subplot is adjusted so that the shear stick density remains the same regardless of size (40 sticks per axis).

In [ ]:
S.plot(subplot=[0.0, -1.0 , 0.0, 1.0])

Shearmaps also have the same coordinate options as Kappamaps.

In [ ]:
S.plot(subplot=[0, 1000, 0, 1000], coords='pixel')

When the subplot is not square, the sampling of shear sticks is held constant rather than the number of sticks so that the image is not distorted.

In [ ]:
S.plot(subplot=[0, 3000, 0, 2000], coords='pixel')

However, the size of the sticks still seems dependent on the size of the x-axis. This makes subplots with a larger y-axis harder to view as seen below. (We need to look further into `numpy.quiver()` to determine how to resolve this.)

In [ ]:
S.plot(fig_size=10, subplot=[0, 2000, 0, 3000], coords='pixel')

## Overlaying shear and convergence

The plotting methods simply use the current axes, allowing simple overlays to be made:   

In [ ]:
K.plot(fig_size=15, subplot=[-0.031, -0.029, -0.0345, -0.033], coords='physical')
S.plot(fig_size=15, subplot=[-0.031, -0.029, -0.0345, -0.033], coords='physical')

In [ ]:
K.plot(fig_size=15, subplot=[-0.0075, -0.005, -0.004, -0.0015], coords='physical')
S.plot(fig_size=15, subplot=[-0.0075, -0.005, -0.004, -0.0015], coords='physical')

If different subplot regions are passed, the `Kappamap` subplot is viewed and only the shear sticks in the overlap area are plotted.

In [ ]:
K.plot(subplot=[1.65, 1.5, -1.65, -1.5])
S.plot(subplot=[1.65, 1.5, -1.65, -1.55])